In [21]:
# Load libraries
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from datetime import datetime, timezone
import errno

In [22]:
def parse_json_file(file_path):
    #open json file
    try:
        with open(file_path, 'r') as read_file:
            data = json.load(read_file)
    except (OSError, IOError) as e: 
       if getattr(e, 'errno', 0) == errno.ENOENT:
          print('json file not found!') # file not found
       raise

    # Parse json file into dataframe
    df_original = json_normalize(data, record_path=['user'], meta=['session_id','unix_timestamp', 'cities'])
    df_user = json_normalize(df_original[0])
    df = pd.concat([df_original, df_user], axis = 1)
    df.drop(columns=[0,'_row'], axis=1, inplace=True)
    df['country'].replace('', 'Unknown', inplace=True)

    df["joining_date"] = pd.to_datetime(df["joining_date"])
    df["unix_timestamp"] = pd.to_datetime(df["unix_timestamp"],unit='s')
    df['access_date'] = df['unix_timestamp'].dt.date
    df['country'] = df['country'].astype('category').cat.as_ordered()
    return df

In [23]:
# For test
file_path = 'city_search.json'
df = parse_json_file(file_path)
df.columns
df.info()
df.head(8)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20022 entries, 0 to 20021
Data columns (total 7 columns):
session_id        20022 non-null object
unix_timestamp    20022 non-null datetime64[ns]
cities            20022 non-null object
country           20022 non-null category
joining_date      20022 non-null datetime64[ns]
user_id           20022 non-null int64
access_date       20022 non-null object
dtypes: category(1), datetime64[ns](2), int64(1), object(3)
memory usage: 958.5+ KB


,session_id,unix_timestamp,cities,country,joining_date,user_id,access_date
0,X061RFWB06K9V,2015-09-17 15:28:28,"New York NY, Newark NJ",UK,2015-03-22,2024,2015-09-17
1,5AZ2X2A9BHH5U,2015-09-04 08:06:31,"New York NY, Jersey City NJ, Philadelphia PA",DE,2015-03-28,2853,2015-09-04
2,SHTB4IYAX4PX6,2015-08-29 10:18:10,San Antonio TX,UK,2015-03-06,10958,2015-08-29
3,JBRB8MZGTX3M4,2015-03-25 07:21:03,Edmonton AB,IT,2015-03-12,7693,2015-03-25
4,YJCMPURC2FL9C,2015-05-02 09:31:07,"Phoenix AZ, Houston TX",UK,2015-02-28,7506,2015-05-02
5,LKOKP80QD6BEO,2015-06-13 12:53:11,San Diego CA,ES,2015-03-04,3743,2015-06-13
6,YOVUIM79SGS5Y,2015-09-25 09:04:47,"Montreal QC, Chicago IL",Unknown,2015-03-02,8831,2015-09-25
7,SHXEY67QWKP9K,2015-05-16 08:48:24,"Calgary AB, New York NY",Unknown,2015-03-16,587,2015-05-16
